In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np 


Using TensorFlow backend.


In [2]:
tokenizer = Tokenizer()

In [3]:
def dataset_preparation(data):

	# basic cleanup
	corpus = data.lower().split("\n")

	# tokenization	
	tokenizer.fit_on_texts(corpus)
	total_words = len(tokenizer.word_index) + 1

	# create input sequences using list of tokens
	input_sequences = []
	for line in corpus:
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(1, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)

	# pad sequences 
	max_sequence_len = max([len(x) for x in input_sequences])
	input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

	# create predictors and label
	predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
	label = ku.to_categorical(label, num_classes=total_words)

	return predictors, label, max_sequence_len, total_words

In [5]:
def create_model(predictors, label, max_sequence_len, total_words):
	
	model = Sequential()
	model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
	model.add(LSTM(150, return_sequences = True))
	# model.add(Dropout(0.2))
	model.add(LSTM(100))
	model.add(Dense(total_words, activation='softmax'))

	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
	model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
	print(model.summary())
	return model 


In [6]:
def generate_text(seed_text, next_words, max_sequence_len):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
		predicted = model.predict_classes(token_list, verbose=0)
		
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text


In [10]:
data = """
If you can make one heap of all your winnings
And risk it on one turn of pitch-and-toss,
And lose, and start again at your beginnings,
And never breathe a word about your loss:
If you can force your heart and nerve and sinew
To serve your turn long after they are gone,
And so hold on when there is nothing in you
Except the Will which says to them: 'Hold on! '

If you can talk with crowds and keep your virtue,
Or walk with Kings- -nor lose the common touch,
If neither foes nor loving friends can hurt you,
If all men count with you, but none too much:
If you can fill the unforgiving minute
With sixty seconds' worth of distance run,
Yours is the Earth and everything that's in it,
And- -which is more- -you'll be a Man, my son!"""

In [11]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)


W1027 11:51:43.248136 15072 deprecation_wrapper.py:119] From C:\installs\Anaconda\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
130/130 [==============================] - 3s 26ms/step - loss: 4.5544 - accuracy: 0.0154
Epoch 2/100
 64/130 [=============>................] - ETA: 0s - loss: 4.5494 - accuracy: 0.0469

C:\installs\Anaconda\envs\tf_gpu\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


130/130 [==============================] - 0s 3ms/step - loss: 4.5470 - accuracy: 0.0538
Epoch 3/100
130/130 [==============================] - 0s 3ms/step - loss: 4.5397 - accuracy: 0.0462
Epoch 4/100
130/130 [==============================] - 0s 3ms/step - loss: 4.5249 - accuracy: 0.0462
Epoch 5/100
130/130 [==============================] - 0s 2ms/step - loss: 4.4834 - accuracy: 0.0462
Epoch 6/100
130/130 [==============================] - 0s 3ms/step - loss: 4.3860 - accuracy: 0.0462
Epoch 7/100
130/130 [==============================] - 0s 2ms/step - loss: 4.3940 - accuracy: 0.0462
Epoch 8/100
130/130 [==============================] - 0s 2ms/step - loss: 4.3439 - accuracy: 0.0538
Epoch 9/100
130/130 [==============================] - 0s 2ms/step - loss: 4.3469 - accuracy: 0.0538
Epoch 10/100
130/130 [==============================] - 0s 2ms/step - loss: 4.3459 - accuracy: 0.0538
Epoch 11/100
130/130 [==============================] - 0s 2ms/step - loss: 4.3203 - accuracy: 0.0538


130/130 [==============================] - 0s 2ms/step - loss: 2.7194 - accuracy: 0.2385
Epoch 83/100
130/130 [==============================] - 0s 2ms/step - loss: 2.6423 - accuracy: 0.3077
Epoch 84/100
130/130 [==============================] - 0s 2ms/step - loss: 2.6451 - accuracy: 0.2846
Epoch 85/100
130/130 [==============================] - 0s 3ms/step - loss: 2.6388 - accuracy: 0.3000
Epoch 86/100
130/130 [==============================] - 0s 2ms/step - loss: 2.5733 - accuracy: 0.3154
Epoch 87/100
130/130 [==============================] - 0s 2ms/step - loss: 2.5375 - accuracy: 0.3385
Epoch 88/100
130/130 [==============================] - 0s 2ms/step - loss: 2.4974 - accuracy: 0.3462
Epoch 89/100
130/130 [==============================] - 0s 2ms/step - loss: 2.4862 - accuracy: 0.3462
Epoch 90/100
130/130 [==============================] - 0s 2ms/step - loss: 2.4685 - accuracy: 0.3615
Epoch 91/100
130/130 [==============================] - 0s 2ms/step - loss: 2.4572 - accuracy: 

In [16]:
print(generate_text("love", 3, max_sequence_len))

love can can can
